In [1]:
# Import libraries
import betfairlightweight
from betfairlightweight import filters
import pandas as pd
import numpy as np
import os
import datetime
import json
from utils import process_runner_books
import pytz
import time
from IPython.display import display

# Project libraries
import constants

# Change this certs path to wherever you're storing your certificates
certs_path = "/home/naing/certs/"
# Change these login details to your own
my_username = constants.USERNAME
my_password = constants.PASSWORD
# my_app_key = constants.API_KEY_DEMO #demo
my_app_key = constants.API_KEY_LIVE #live

trading = betfairlightweight.APIClient(username=my_username,
                                       password=my_password,
                                       app_key=my_app_key,
                                       certs=certs_path)

trading.login()

<LoginResource>

In [2]:
#######################################
# List the cleared orders
#######################################
myRaceID = '1.200715893'
trading.betting.list_current_orders(
    market_ids=[myRaceID]).__dict__


{'streaming_unique_id': None,
 'streaming_update': None,
 'streaming_snap': False,
 'publish_time': None,
 'elapsed_time': 0.41095900535583496,
 '_datetime_created': datetime.datetime(2022, 7, 3, 12, 22, 29, 662606),
 '_datetime_updated': datetime.datetime(2022, 7, 3, 12, 22, 29, 662606),
 '_data': {'currentOrders': [], 'moreAvailable': False},
 'more_available': False,
 'orders': [],
 'matches': []}

In [27]:
def choose_lay_option_bf(price_filter, lay_selection_index):
    # Request market books
    market_books = trading.betting.list_market_book(
        market_ids=[myRaceID],
        price_projection=price_filter
    )

    # Grab the first market book from the returned list as we only requested one market 
    market_book = market_books[0]
    runners_df = utils.process_runner_books(market_book.runners)
    
    display(runners_df)

    #TODO: extract a list of IDs for the lay options
    lay_options_ids = []
    for obj in market_book.runners:
        lay_options_ids.append(obj.selection_id)
        
    # if debug:
    #     print(lay_options_ids)
    
    lay_selection_id = lay_options_ids[lay_selection_index]
    print("lay_selection_id = " + str(lay_selection_id))
    fav_price = runners_df.loc[runners_df['Best Lay Price'].idxmin(), 'Best Lay Price']

    return lay_selection_id, fav_price


import utils

fullyMatched_flag = False

while fullyMatched_flag == False:
    ## get the current order
    current_orders = trading.betting.list_current_orders(
        market_ids=[myRaceID])
    single_current_order = current_orders._data['currentOrders'][0]
    print(single_current_order)

    unmatched_size = single_current_order['sizeRemaining']
    print('unmatched size = ', unmatched_size)

    ## if there's unmatched order, replace it with new price, else set the fullyMatched flag
    if float(unmatched_size) != 0:
        print("Order is not fully matched yet")

        # Get the new best Lay price
        # price_filter = filters.price_projection(
        #     price_data=['EX_BEST_OFFERS']
        # )
        # lay_selection_id, fav_price = choose_lay_option_bf(price_filter, 0)

        ## Replace the unmatched order with the new price
        replace_instructions_filter = filters.replace_instruction(
            bet_id= single_current_order['betId'],
            new_price= '6' #TODO
        )
        print(replace_instructions_filter)

        newOrder = trading.betting.replace_orders(
                market_id = myRaceID,
                customer_ref='Naing_maker_NEW',
                instructions=[replace_instructions_filter]
            )

        print("full order matched deliberately")
        # fullyMatched_flag = True
        time.sleep(5)
    else:
        print("FUll order matched")
        fullyMatched_flag = True





{'betId': '273046589972', 'marketId': '1.200715847', 'selectionId': 41419585, 'handicap': 0.0, 'priceSize': {'price': 6.0, 'size': 2.0}, 'bspLiability': 0.0, 'side': 'LAY', 'status': 'EXECUTABLE', 'persistenceType': 'LAPSE', 'orderType': 'LIMIT', 'placedDate': '2022-07-03T07:15:39.000Z', 'averagePriceMatched': 0.0, 'sizeMatched': 0.0, 'sizeRemaining': 2.0, 'sizeLapsed': 0.0, 'sizeCancelled': 0.0, 'sizeVoided': 0.0, 'regulatorCode': 'MALTA LOTTERIES AND GAMBLING AUTHORITY', 'customerStrategyRef': 'Naing_maker'}
unmatched size =  2.0
Order is not fully matched yet
{'betId': '273046589972', 'newPrice': '6'}
full order matched deliberately
{'betId': '273046589972', 'marketId': '1.200715847', 'selectionId': 41419585, 'handicap': 0.0, 'priceSize': {'price': 6.0, 'size': 2.0}, 'bspLiability': 0.0, 'side': 'LAY', 'status': 'EXECUTABLE', 'persistenceType': 'LAPSE', 'orderType': 'LIMIT', 'placedDate': '2022-07-03T07:15:39.000Z', 'averagePriceMatched': 0.0, 'sizeMatched': 0.0, 'sizeRemaining': 2.

KeyboardInterrupt: 

In [3]:
# %%
#######################################
# Choose order type (limit order or market_on_close order - choose the latter)
#######################################
order_filter = filters.limit_order(
    price='5',
    persistence_type='LAPSE',
    bet_target_type='PAYOUT',
    bet_target_size='10' 
)
print(order_filter)

#######################################
# Create a place instruction filter
#######################################
instructions_filter = filters.place_instruction(
    selection_id = '41419585',
    side="LAY",
    order_type = "LIMIT", # fixed price order
    limit_order=order_filter
)
print(instructions_filter)


# %%
#######################################
# Place the order
#######################################
order = trading.betting.place_orders(
    market_id = myRaceID,
    customer_strategy_ref='Naing_maker',
    instructions=[instructions_filter]
)

{'price': '5', 'persistenceType': 'LAPSE', 'betTargetType': 'PAYOUT', 'betTargetSize': '10'}
{'orderType': 'LIMIT', 'selectionId': '41419585', 'side': 'LAY', 'limitOrder': {'price': '5', 'persistenceType': 'LAPSE', 'betTargetType': 'PAYOUT', 'betTargetSize': '10'}}
